In [1]:
# fake new detection using spacy word embedding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [3]:
fake_df = pd.read_csv("fake new detection\Fake.csv")
true_df = pd.read_csv('fake new detection\True.csv')
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
fake_df["is_fake"] = 1

In [6]:
true_df["is_fake"]= 0

In [8]:
fake_df.head()
true_df.head()

,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [11]:
df = pd.concat([fake_df,true_df],axis=0)  
df.sample(10)

,title,text,subject,date,is_fake
19873,Australia's Turnbull defends 'religious freedo...,SYDNEY (Reuters) - Australia s Prime Minister ...,worldnews,"September 16, 2017",0
6979,Muslim Woman Who Turned In Paris Terrorist Sh...,"Abdelhamid Abaaoud, the planner of the terrori...",News,"April 10, 2016",1
10253,SARAH HUCKABEE SANDERS MOCKS WH PRESS: Lists A...,Sarah Huckabee Sanders let the White House pre...,politics,"Aug 1, 2017",1
17845,WATCH: INTOLERANT GAY COFFEE SHOP Owner Scream...,Remember the time when a gay couple destroyed ...,left-news,"Oct 8, 2017",1
8405,"In attacking Clinton foundation, Trump seeks a...",WASHINGTON (Reuters) - After struggling to fin...,politicsNews,"August 23, 2016",0
6983,Senior senators want to amend Saudi September ...,WASHINGTON (Reuters) - Two senior U.S. senator...,politicsNews,"November 30, 2016",0
21044,Homeless Man Dies Next To 4-STAR HOTEL…Your Bl...,LIBERAL COMPASSION: This story is a perfect ex...,left-news,"Feb 2, 2016",1
20241,Factbox: About 5.8 million without power in U....,(Reuters) - Power outages from Hurricane Irma ...,worldnews,"September 12, 2017",0
21375,GERMAN VOLUNTEERS HOLD WELCOME RALLY: Applaud ...,"Say goodbye to your economy, traditions and cu...",left-news,"Oct 31, 2015",1
15491,Japan nuclear regulator says restart approval ...,TOKYO (Reuters) - The pace of approvals for nu...,worldnews,"November 7, 2017",0


In [13]:
df.is_fake.value_counts()

1    23481
0    21417
Name: is_fake, dtype: int64

In [14]:
import spacy 
nlp = spacy.load("en_core_web_lg")

In [15]:
doc = nlp("")
doc.vector.shape

(300,)

In [17]:
# adding vector to df
df["text_vector"] = df["text"].apply(lambda x: nlp(x).vector)
df.head()

,title,text,subject,date,is_fake,text_vector
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1,"[-1.6387042, -0.027837504, -1.6619873, -0.2017..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,"[-1.9558355, 0.5587237, -2.036446, 0.40390933,..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1,"[-1.9058484, -0.04223674, -1.7389339, -0.02855..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1,"[-1.6265213, 0.14619705, -0.5877981, 0.0992525..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1,"[-2.1327376, 1.1017615, -2.4795847, -0.0065187..."


In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df["text_vector"],df["is_fake"],test_size=0.2,stratify=df["is_fake"],random_state=2022)


In [19]:
# x_train is 1d numpy array in which it has multiple numpy array so we have to convert it into 2d 
x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train_2d)
x_test_scaled = scaler.transform(x_test_2d)

model = MultinomialNB()
model.fit(x_train_scaled,y_train)
# multimonial nb don't accept negetive values remove it using scalling

MultinomialNB()

In [21]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test_scaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86      4284
           1       0.90      0.84      0.87      4696

    accuracy                           0.87      8980
   macro avg       0.87      0.87      0.87      8980
weighted avg       0.87      0.87      0.87      8980



In [22]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(x_train_scaled,y_train)

KNeighborsClassifier()

In [23]:
y_pred = model.predict(x_test_scaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4284
           1       0.97      0.97      0.97      4696

    accuracy                           0.97      8980
   macro avg       0.97      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980



In [27]:
df.to_csv("fake new detection\\true_fake.csv",index=False)